# Progetto di Foundations of Computer Science

#### Caterina Pisani
#### Matricola: 853058
#### Mail istituzionale: c.pisani6@campus.unimib.it

L'intero progetto è basato sull'analisi dei dati provenienti dal dataset Kiva.snapshot, reperibile al link: https://www.kaggle.com/gaborfodor/additional-kiva-snapshot


**Kiva Microfunds** è un network globale che, tramite il portale  http://www.kiva.org, promuove azioni di microcredito in Paesi disagiati attraverso la raccolta di fondi via internet. 
Nella pratica, il sito raccoglie una serie di profili di piccoli imprenditori in modo da metterli in contatto con i contributori, che sono liberi di visionare ogni profilo e decidere quale progetto finanziare e con quale importo.


Il dataset a nostra disposizione fornisce informazioni riguardanti circa 1.4 milioni di prestiti e 2.3 milioni di prestatori e si compone di 7 tabelle: loans.csv, lenders.csv, loans_lenders.csv, country_stats.csv, GEconV4.csv, locations.csv, loan_coords.csv.

In [1]:
#Importo librerie
import pandas as pd
import numpy as np
import dask.dataframe as dd
from IPython.display import display
import datetime as  dt
import jupyter_contrib_nbextensions
import mysql.connector
from tabulate import tabulate

# 1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender

In [2]:
loans_lenders=dd.read_csv('/Users/caterinapisani/Progetto FCS/KIVA/loans_lenders.csv')

In [3]:
loans_lenders.compute()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
138359,678999,"michael43411218, carol5987, gooddogg1, chris41..."
138360,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
138361,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
138362,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


In [4]:
#Controllo se sono presenti valori Nan
loans_lenders.isna().sum().compute()

loan_id    0
lenders    0
dtype: int64

In [5]:
#Controllo se ci sono duplicati nella colonna loan_id
len(loans_lenders['loan_id'].unique().compute())

1387432

Ora posso procedere con la normalizzazione della tabella. Siccome le stringhe della colonna "lenders" sono separate da virgola, eseguo prima di tutto  una split. Dopodichè posso applicare la funzione "explode", che letteralmente fa "esplodere" le stringhe su più righe, così da avere la tabella in forma normale. 

In [6]:
norm_loans_lenders = loans_lenders.assign(lenders=loans_lenders["lenders"].str.split(",")).explode("lenders").reset_index(drop=True)

In [7]:
norm_loans_lenders.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [8]:
len(norm_loans_lenders)

28293931

Si noti che normalizzando il df, le righe sono aumentate di circa 20 volte.

# 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [9]:
#Leggo il file e includo un'opzione per poter visualizzare tutte le colonne 
loans=dd.read_csv('/Users/caterinapisani/Progetto FCS/KIVA/loans.csv')
pd.set_option('display.max_columns', None)

In [10]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [11]:
#Voglio sapere il tipo di dati della tabella, ed in particolare noto che "planned_expiration_time"
#e "disburse_time" sono considerati oggetti, non sono ancora in formato datetime.
loans.dtypes

loan_id                              int64
loan_name                           object
original_language                   object
description                         object
description_translated              object
funded_amount                      float64
loan_amount                        float64
status                              object
activity_name                       object
sector_name                         object
loan_use                            object
country_code                        object
country_name                        object
town_name                           object
currency_policy                     object
currency_exchange_coverage_rate    float64
currency                            object
partner_id                         float64
posted_time                         object
planned_expiration_time             object
disburse_time                       object
raised_time                         object
lender_term                        float64
num_lenders

In [12]:
#Sfrutto l'opzione "parse_dates" della read.csv per trasformare in formato datetime le colonne di interesse
loans=dd.read_csv('/Users/caterinapisani/Progetto FCS/KIVA/loans.csv', parse_dates=['planned_expiration_time', 'disburse_time'])

In [13]:
#Creo un dataframe con solo le colonne di interesse 
#Rimuovo i valori nulli
#Mantengo solo i casi in cui planned_expiration_time>disburse_time perchè non ha senso un prestito che si estingue
#prima della scadenza.
#Potrei invertire le date ma preferisco essere conservativa ed evitare di modificare erroneamente le proprietà del dataset
loans_dates=loans[['loan_id','disburse_time', 'planned_expiration_time', 'loan_amount']].dropna()

In [14]:
loans_dates = loans_dates[loans_dates["planned_expiration_time"] > loans_dates["disburse_time"]]

In [15]:
loans_dates.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,400.0
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,625.0
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0


In [16]:
#Controllo che il tipo di dati sia effettivamente cambiato
loans_dates.dtypes

loan_id                                  int64
disburse_time              datetime64[ns, UTC]
planned_expiration_time    datetime64[ns, UTC]
loan_amount                            float64
dtype: object

In [17]:
#Ora posso creare una nuova colonna "duration" data dalla differenza tra planned expiration time e disburse time

loans_duration = loans_dates.assign(duration=loans['planned_expiration_time']-loans['disburse_time'])

In [18]:
loans_duration.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount,duration
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0,53 days 19:30:06
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0,96 days 14:25:07
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,400.0,37 days 13:10:05
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,625.0,34 days 19:10:02
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0,57 days 22:10:02


In [19]:
loans_duration.tail()

,loan_id,disburse_time,planned_expiration_time,loan_amount,duration
27092,988180,2015-11-23 08:00:00+00:00,2016-01-02 01:00:03+00:00,400.0,39 days 17:00:03
27093,988213,2015-11-24 08:00:00+00:00,2016-01-02 16:40:07+00:00,300.0,39 days 08:40:07
27094,989109,2015-11-13 08:00:00+00:00,2016-01-03 22:20:04+00:00,2425.0,51 days 14:20:04
27095,989143,2015-11-03 08:00:00+00:00,2016-01-05 08:50:02+00:00,100.0,63 days 00:50:02
27096,989240,2015-11-03 08:00:00+00:00,2016-01-03 20:50:06+00:00,175.0,61 days 12:50:06


In [20]:
#Confronto la somma di Nan in diburse_time e planned_expiration_time con i Nan della colonna duration
Tot_Na= loans.disburse_time.isna().sum()+loans.planned_expiration_time.isna().sum()
Tot_Na.compute()

374647

In [21]:
Na_duration = loans_duration.duration.isna().sum().compute()
Na_duration

0

La colonna duration presenta due valori Nan in meno, questo significa solo che in 2 casi i valori Nan erano presenti sia in Disburse_time che in Planned_expiration_time per lo stesso loan_id. 

### Per lo svolgimento dei successivi punti del progetto, verrà adottata la libreria "mysql.connector"

Dal punto 3 al punto 7 le richieste consistono in vere e proprie query che possono essere gestite efficaciemente con linguaggio SQL.

In [22]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="ProgettoFCS",
  database="kivapisani"
)

mycursor = mydb.cursor()

# 3. Find the lenders that have funded at least twice.

In [23]:
#Join tra loans e norm_loans_lenders
#Impongo status=funded e funded>1
#Group by lend_name 
query_1 = """SELECT lend_name, count(status) as funded
FROM norm_loans_lenders LEFT JOIN loans ON loans.loan_id=norm_loans_lenders.loan_id
WHERE status='funded'
GROUP BY lend_name
HAVING funded>1
LIMIT 10"""

In [24]:
mycursor.execute(query_1)
myresult = mycursor.fetchall()
print(tabulate(myresult, headers=['Lend_name', 'Funded'], tablefmt='pretty'))

+--------------------+--------+
|     Lend_name      | Funded |
+--------------------+--------+
|   bernadette6835   |   27   |
|      jean2544      |  417   |
|        geko        |   33   |
|      lynn9800      |   4    |
|      brad4706      |  130   |
|       muc888       |  913   |
|     dougal1825     |  1141  |
| jensdamsgaardvanar |  7815  |
|     maeve9455      |   8    |
|      toni3909      |   34   |
+--------------------+--------+


# 4. For each country, compute how many loans have involved that country as borrowers.

In [25]:
#Count dei loan_id
#Group by country_name
query_2 = """SELECT country_name, count(loan_id) as count
FROM loans
GROUP BY country_name
ORDER BY country_name"""

In [26]:
mycursor.execute(query_2)
myresult = mycursor.fetchall()
print(tabulate(myresult, headers=['Country_name', 'Count'], tablefmt='pretty'))

+--------------------------------------+--------+
|             Country_name             | Count  |
+--------------------------------------+--------+
|             Afghanistan              |  2337  |
|               Albania                |  3075  |
|               Armenia                | 13952  |
|              Azerbaijan              | 10172  |
|                Belize                |  218   |
|                Benin                 |  5946  |
|                Bhutan                |   2    |
|               Bolivia                | 25250  |
|        Bosnia and Herzegovina        |  608   |
|               Botswana               |   1    |
|                Brazil                |  482   |
|               Bulgaria               |  296   |
|             Burkina Faso             |  3489  |
|               Burundi                |  1727  |
|               Cambodia               | 79701  |
|               Cameroon               |  5085  |
|                Canada                |   1    |


# 5. For each country, compute the overall amount of money borrowed.

In [27]:
#Sum di loan_amount -> money_borrowed
#Group by country_name
query_3 = """Select country_name, sum(loan_amount) as money_borrowed
from loans
group by country_name
order by country_name"""

In [28]:
mycursor.execute(query_3)
myresult = mycursor.fetchall()
print ((tabulate(myresult, headers=['Country_name', 'Money_borrowed'], tablefmt='pretty')))

+--------------------------------------+----------------+
|             Country_name             | Money_borrowed |
+--------------------------------------+----------------+
|             Afghanistan              |    1967950     |
|               Albania                |    4307350     |
|               Armenia                |    22950475    |
|              Azerbaijan              |    14784625    |
|                Belize                |     150175     |
|                Benin                 |    3865825     |
|                Bhutan                |     20000      |
|               Bolivia                |    44226725    |
|        Bosnia and Herzegovina        |     477250     |
|               Botswana               |      8000      |
|                Brazil                |    1192325     |
|               Bulgaria               |     375300     |
|             Burkina Faso             |    4085200     |
|               Burundi                |    5233450     |
|             

# 6. Like the previous point, but expressed as a percentage of the overall amount lent.

In [29]:
#Per trovare la percentuale del denaro preso in prestito (money_borrowed) sul totale del denaro prestato (loan_amount)
#Inserisco nella select la sum del loan_amount 
#Inserisco nella select il rapporto che voglio trovare: al numeratore ho la sum di loan_amount (ovvero l'importo totale dei loans) moltiplicata per 100,
#al denominatore ho una nested query per calcolare la somma di loan_amount per ogni Paese (country_name)
#Group by country name
query_4 = """Select distinct country_name, 
sum(loan_amount),
sum(loan_amount)*100/(select sum(loan_amount) from loans) as perc_money_borrowed
from loans
group by country_name
order by country_name"""

In [30]:
mycursor.execute(query_4)
myresult = mycursor.fetchall()
print ((tabulate(myresult, headers=['Country_name','Money_borrowed', 'Perc_money_borrowed'], tablefmt='pretty')))

+--------------------------------------+----------------+---------------------+
|             Country_name             | Money_borrowed | Perc_money_borrowed |
+--------------------------------------+----------------+---------------------+
|             Afghanistan              |    1967950     |       0.1666        |
|               Albania                |    4307350     |       0.3646        |
|               Armenia                |    22950475    |       1.9426        |
|              Azerbaijan              |    14784625    |       1.2514        |
|                Belize                |     150175     |       0.0127        |
|                Benin                 |    3865825     |       0.3272        |
|                Bhutan                |     20000      |       0.0017        |
|               Bolivia                |    44226725    |       3.7435        |
|        Bosnia and Herzegovina        |     477250     |       0.0404        |
|               Botswana               |

# 7. Like the three previous points, but split for each year (with respect to disburse time ).

In [31]:
#Eseguo gli stessi conti della precedente richiesta
#Aggiungo una clausola where dove richiedo di estrarre solo le righe in cui disburse_time sia minore di planned_expiration_time
#Escludo anche le percentuali pari a 0
query_5 = """Select country_name, 
EXTRACT(YEAR from disburse_time) AS year, 
sum(loan_amount) as money_borrowed,
sum(loan_amount)/(select sum(loan_amount) from loans) as perc_money_borrowed
from loans
where disburse_time < planned_expiration_time
group by country_name, year
having perc_money_borrowed > 0.00009
ORDER BY country_name
LIMIT 50"""

In [32]:
mycursor.execute(query_5)
myresult = mycursor.fetchall()
print ((tabulate(myresult, headers=['Country_name', 'Disburse_time', 'Money_borrowed', 'Perc_money_borrowed'], tablefmt='pretty')))

+--------------+---------------+----------------+---------------------+
| Country_name | Disburse_time | Money_borrowed | Perc_money_borrowed |
+--------------+---------------+----------------+---------------------+
|   Albania    |     2012      |     413000     |       0.0003        |
|   Albania    |     2013      |     786950     |       0.0007        |
|   Albania    |     2014      |     902550     |       0.0008        |
|   Albania    |     2015      |     844375     |       0.0007        |
|   Albania    |     2016      |     639025     |       0.0005        |
|   Albania    |     2017      |     721450     |       0.0006        |
|   Armenia    |     2011      |     273300     |       0.0002        |
|   Armenia    |     2012      |    2048075     |       0.0017        |
|   Armenia    |     2013      |    4131300     |       0.0035        |
|   Armenia    |     2014      |    3280025     |       0.0028        |
|   Armenia    |     2015      |    3519825     |       0.0030  

# 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


Da questo punto e fino alla fine del progetto smetto di usare Mysql connector, le query richieste dai punti successive risulterebbero troppo onerose computazionalmente e si perderebbe il vantaggio avuto nei punti precedenti. 

In [33]:
#comincio a ricavare il numero di lenders per ogni loan_id ed eseguo una group by per loan_id
lenders_per_loan = norm_loans_lenders.groupby("loan_id").count().reset_index().rename(columns={"lenders" : "lenders_count"})

In [34]:
lenders_per_loan.head()

,loan_id,lenders_count
0,84,3
1,89,4
2,102,6
3,108,3
4,121,4


In [35]:
# eseguo una merge con loans per avere anche le informazioni riguardanti il loan_amount
lenders_loan_amount = dd.merge(lenders_per_loan, loans, on="loan_id")[['loan_id','lenders_count','loan_amount']]

In [36]:
#Aggiungo una colonna in cui calcolo il loan_amount per lender, assumendo che tutti abbiano contribuito con lo stesso ammontare

lenders_loan_amount["amount_per_person"] = lenders_loan_amount["loan_amount"] / lenders_loan_amount["lenders_count"]

In [37]:
lenders_loan_amount.head()

,loan_id,lenders_count,loan_amount,amount_per_person
0,13558,3,200.0,66.666667
1,87422,22,900.0,40.909091
2,87425,21,800.0,38.095238
3,87446,35,1000.0,28.571429
4,87461,19,600.0,31.578947


In [38]:
#Ora unisco norm_loans_lenders e lenders_loan_amount in modo da avere un df con "amount_per_person" associato ad ogni lender

loans_lenders_merged = dd.merge(norm_loans_lenders, lenders_loan_amount, on="loan_id", how="left")

In [39]:
#Ora raggruppo per lender e sommo, ottenendo il totale prestato da ogni lender.

lenders_overall_lent = loans_lenders_merged.groupby("lenders")["amount_per_person"].sum().to_frame().reset_index()

In [40]:
#Converto il df lenders_overall_lent in un df pandas perchè tornerà utile nei punti successivi
lenders_overall_lent=lenders_overall_lent.compute()

In [41]:
lenders_overall_lent.head()

,lenders,amount_per_person
0,000,1672.618411
1,00000,1380.693644
2,0245597,33.333333
3,0326lsw,1794.927610
4,0367630,31.250000


# 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


In [42]:
lenders = pd.read_csv('/Users/caterinapisani/Progetto FCS/KIVA/lenders.csv')

In [43]:
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [44]:
lenders.isna().sum()

permanent_name             0
display_name            2768
city                 1619306
state                1713481
country_code         1458635
member_since               0
occupation           1844514
loan_because         2174852
loan_purchase_num     894281
invited_by           1852349
num_invited                0
dtype: int64

Come riportato nel testo della richiesta, vi sono molti valori mancanti nella colonna "country_code" e scegliamo di assumere che la distribuzione tra i Paesi dai valori mancanti sia la stessa di quelli in cui compare un valore. Dunque, il primo problema di cui occuparsi è quello del calcolo della distribuzione.  

In [45]:
#estraggo i country code non nulli e creo "lenders_notnull"
lenders_notnull = lenders.loc[lenders["country_code"].notnull()].reset_index()
lenders_notnull

,index,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,16,naresh2074,Naresh,NaN,NaN,US,1461303119,NaN,NaN,3.0,NaN,0
1,31,christina27976796,Christina,NaN,NaN,US,1461303341,NaN,NaN,1.0,Peter Tan,0
2,37,vikas1098,Vikas,Bengaluru,NaN,IN,1461301846,Software Engineer,NaN,2.0,NaN,0
3,39,qian1385,Qian,NaN,NaN,US,1461302036,NaN,NaN,2.0,NaN,0
4,42,xigg8769,Darla,Madison,NaN,US,1461302503,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
890534,2349158,rakhi,Rakhi,New York,New York,US,1342100607,Student,I care.,4.0,NaN,0
890535,2349159,vicki5374,Vicki,Austin,Texas,US,1342101743,NaN,NaN,1.0,Vicki,1
890536,2349161,jennifer5879,Jennifer,Riverview,New Brunswick,CA,1342092207,NaN,NaN,1.0,Beverly,1
890537,2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0


In [46]:
tot_notnull_lenders = len(lenders_notnull.index)
tot_notnull_lenders

890539

In [47]:
#estraggo i country code nulli e creo "lenders_null"
lenders_null = lenders.loc[lenders["country_code"].isnull()].reset_index()

In [48]:
#Calcolo la distribuzione delle nazioni nel dataframe lenders_notnull: innanzitutto eseguo una group_by 
#e una count sul country_code per ottenere il num di lender per country 

lenders_per_country = lenders_notnull[["index", "country_code"]].groupby("country_code").count().reset_index().rename(columns = {"index":"n_lenders"})
lenders_per_country.head()

,country_code,n_lenders
0,AD,15
1,AE,1043
2,AF,228
3,AG,8
4,AI,4


In [49]:
#secondariamente calcolo la percentuale del num di lenders per country sul totale dei lenders non nulli

lenders_per_country["percentage"] = lenders_per_country["n_lenders"]/tot_notnull_lenders*100

lenders_per_country.head()

,country_code,n_lenders,percentage
0,AD,15,0.001684
1,AE,1043,0.117120
2,AF,228,0.025602
3,AG,8,0.000898
4,AI,4,0.000449


In [50]:
somma_perc = lenders_per_country["percentage"].sum()
somma_perc

100.0

In [51]:
#Infine, normalizzo tutte le percentuali dividendole per la loro somma in modo che il totale sia pari a 1 e non si ottengano errori

lenders_per_country["percentage"] /= lenders_per_country["percentage"].sum()
somma_perc = lenders_per_country["percentage"].sum()
somma_perc

1.0

In [52]:
#Ora posso riempire il dataframe lenders_null in modo che abbia la stessa distribuzione di lenders_notnull.
#Per fare questo uso la funzione np.random.choice (inserendo come seed '1803', orario corrente)
#In questo modo assegno la nazionalità in modo randomico e non sistematico, ottenendo la stessa distribuzione del df non nullo

np.random.seed(1803)
lenders_null["country_code"] = np.random.choice(lenders_per_country["country_code"], size=len(lenders_null.index), p = lenders_per_country["percentage"])

lenders_null

,index,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,0,qian3013,Qian,NaN,NaN,CA,1461300457,NaN,NaN,1.0,NaN,0
1,1,reena6733,Reena,NaN,NaN,US,1461300634,NaN,NaN,9.0,NaN,0
2,2,mai5982,Mai,NaN,NaN,US,1461300853,NaN,NaN,NaN,NaN,0
3,3,andrew86079135,Andrew,NaN,NaN,GB,1461301091,NaN,NaN,5.0,Peter Tan,0
4,4,nguyen6962,Nguyen,NaN,NaN,US,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1458630,2349167,todd5695,Todd,NaN,NaN,US,1342101461,NaN,NaN,NaN,NaN,0
1458631,2349168,kate40761039,Kate,NaN,NaN,US,1342096938,NaN,NaN,1.0,Irina,0
1458632,2349169,janet7309,Janet,NaN,NaN,US,1342097163,NaN,NaN,NaN,NaN,0
1458633,2349170,pj4198,NaN,NaN,NaN,US,1342097515,NaN,NaN,NaN,NaN,0


In [53]:
#Inserisco nel df "lenders" originario i df lenders_notnull e lenders_null in cui non compaiono più country_code nulli
lenders = pd.concat([lenders_notnull, lenders_null]).drop(columns="index")

In [54]:
lenders.isna().sum()

permanent_name             0
display_name            2768
city                 1619306
state                1713481
country_code               0
member_since               0
occupation           1844514
loan_because         2174852
loan_purchase_num     894281
invited_by           1852349
num_invited                0
dtype: int64

## Compute the difference between the overall amount of money lent and the overall amount of money borrowed.

Ora che il df lenders è sistemato, posso procedere con la richiesta.

#### Overall amount of money borrowed 

In [55]:
#Prima di tutto creo un df con il tot money borrowed per ogni nazione eseguendo una group_by sul country_code e una sum dei loan_amount
tot_borr_country = loans.groupby("country_code")["loan_amount"].sum().reset_index().rename(columns={"loan_amount" : "borrowed_amount"})

In [56]:
tot_borr_country.head()

,country_code,borrowed_amount
0,AF,1967950.0
1,AL,4307350.0
2,AM,22950475.0
3,AZ,14784625.0
4,BA,477250.0


Il df "tot_borr_country" è ancora dask, poichè coinvolge il df loans che viene gestito più efficaciemente in questo modo.

#### Overall amount of money lent

In [57]:
#Ora ho bisogno dello stesso df ma con il tot money lent per ogni nazione 
#Il df lenders mi fornisce informazioni sulla nazionalità dei lenders, mentre il df lenders_overall_lent trovato
#al punto 8 comprende il loan_amount per ogni lender. Dunque, innanzitutto eseguo una merge tra questi df

tot_lent_country1 = pd.merge(lenders_overall_lent, lenders, left_on= 'lenders', right_on='permanent_name').drop(columns='permanent_name')

In [58]:
tot_lent_country1

,lenders,amount_per_person,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,0221581,26.785714,022,NaN,NaN,US,1293534673,NaN,NaN,3.0,NaN,0
1,0589889,58.562500,058,NaN,NaN,US,1293552706,NaN,NaN,4.0,NaN,0
2,0648612,28.787879,064,NaN,NaN,US,1294408211,NaN,NaN,4.0,NaN,0
3,0819212,1947.679376,Robert,Munich,NaN,DE,1294410814,Banker,NaN,197.0,NaN,0
4,100ofhumanity1199,72867.411726,100% of Humanity,NaN,NaN,US,1443473590,NaN,NaN,7662.0,Bryan,2
...,...,...,...,...,...,...,...,...,...,...,...,...
287564,zayd3932,50.000000,Zayd,NaN,NaN,US,1437031822,NaN,NaN,12.0,NaN,0
287565,zbjwt2383,40.909091,九星,NaN,NaN,US,1439378568,NaN,NaN,NaN,Kevin,0
287566,zinniya,75.000000,Barbara,NaN,NaN,CA,1388502203,NaN,NaN,6.0,NaN,0
287567,zoe7680,62.500000,Zoe,Seattle,WA,US,1327871771,NaN,NaN,9.0,Tom,1


In [59]:
#ora posso eseguire una group_by sul country_code e una sum dell'amount_per_person per ottenere tot_lent_country
tot_lent_country=tot_lent_country1.groupby("country_code")["amount_per_person"].sum().reset_index().rename(columns={"amount_per_person" : "lent_amount"})

In [60]:
tot_lent_country.head()

,country_code,lent_amount
0,AD,268.510615
1,AE,145825.810271
2,AF,8614.479897
3,AG,75.000000
4,AI,227.083333


In [61]:
#Ora unisco i due df tot_borr_country e tot_lent_country
country_borr_lent= dd.merge(tot_borr_country,tot_lent_country, on='country_code')

Una merge tra un df dask e un df pandas viene effettuata con comando dask e il df risultante è anch'esso dask, lo convertirò successivamente in pandas.

In [62]:
country_borr_lent.head()

,country_code,borrowed_amount,lent_amount
0,AF,1967950.0,8614.479897
1,AL,4307350.0,1037.850738
2,AM,22950475.0,1740.319733
3,AZ,14784625.0,753.017777
4,BA,477250.0,6129.969938


In [63]:
#creo una colonna per la differenza tra lent_amount e borrowed_amount
country_borr_lent['difference']= (country_borr_lent['lent_amount']-country_borr_lent['borrowed_amount'])


In [64]:
#Converto il df country_borr_lent in un df pandas perchè tornerà utile nei punti successivi
country_borr_lent=country_borr_lent.compute()

In [65]:
country_borr_lent.head()

,country_code,borrowed_amount,lent_amount,difference
0,AF,1967950.0,8614.479897,-1.959336e+06
1,AL,4307350.0,1037.850738,-4.306312e+06
2,AM,22950475.0,1740.319733,-2.294873e+07
3,AZ,14784625.0,753.017777,-1.478387e+07
4,BA,477250.0,6129.969938,-4.711200e+05


# 10. Which country has the highest ratio between the difference computed at the previous point and the population?

In [66]:
#carico il df country_stats perchè mi fornisce informazioni riguardo alla popolazione
country_stats= pd.read_csv('/Users/caterinapisani/Progetto FCS/KIVA/country_stats.csv')
country_stats

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,Somalia,SO,SOM,Africa,Eastern Africa,14742523,NaN,NaN,NaN,NaN,NaN,NaN,Somalia
170,Central African Republic,CF,CAF,Africa,Middle Africa,4659080,NaN,0.352440,51.458,7.098980,4.230000,587.473961,Central African Republic
171,Samoa,WS,WSM,Oceania,Polynesia,196440,NaN,0.702000,NaN,NaN,NaN,NaN,Samoa
172,Palestine,PS,PS,Asia,Western Asia,4920724,NaN,0.677000,NaN,NaN,NaN,NaN,Palestine


In [67]:
#unisco country_stats con il df creato al punto 9 country_borr_lent
country_borr_lent  = pd.merge(country_borr_lent, country_stats[["country_code","country_name", "population"]], on="country_code")


In [68]:
#calcolo prima di tutto il ratio, ovvero il rapporto della differenza tra lent_amount e borrowed_amount e la popolazione totale, e creo una colonna apposita
country_borr_lent['ratio']=country_borr_lent['difference']/country_borr_lent['population']
country_borr_lent

,country_code,borrowed_amount,lent_amount,difference,country_name,population,ratio
0,AF,1967950.0,8.614480e+03,-1.959336e+06,Afghanistan,35530081,-0.055146
1,AL,4307350.0,1.037851e+03,-4.306312e+06,Albania,2930187,-1.469637
2,AM,22950475.0,1.740320e+03,-2.294873e+07,Armenia,2930450,-7.831130
3,AZ,14784625.0,7.530178e+02,-1.478387e+07,Azerbaijan,9827589,-1.504323
4,BA,477250.0,6.129970e+03,-4.711200e+05,Bosnia and Herzegovina,3507017,-0.134336
...,...,...,...,...,...,...,...
81,CA,50000.0,5.724080e+06,5.674080e+06,Canada,36624199,0.154927
82,BW,8000.0,4.214332e+02,-7.578567e+03,Botswana,2291661,-0.003307
83,BT,20000.0,3.319352e+02,-1.966806e+04,Bhutan,807610,-0.024353
84,MR,15000.0,3.441176e+01,-1.496559e+04,Mauritania,4420184,-0.003386


In [69]:
#trovo la nazione dove il ratio è massimo
country_borr_lent[country_borr_lent['ratio']==country_borr_lent['ratio'].max()]

,country_code,borrowed_amount,lent_amount,difference,country_name,population,ratio
81,CA,50000.0,5.724080e+06,5.674080e+06,Canada,36624199,0.154927


# 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [70]:
#Creo una colonna in cui inserire la popolazione sopra la poverty  line
country_borr_lent['population_above_poverty_line'] = country_stats['population'] * (1 - (country_stats['population_below_poverty_line'] / 100))

In [71]:
country_borr_lent

,country_code,borrowed_amount,lent_amount,difference,country_name,population,ratio,population_above_poverty_line
0,AF,1967950.0,8.614480e+03,-1.959336e+06,Afghanistan,35530081,-0.055146,1.045900e+09
1,AL,4307350.0,1.037851e+03,-4.306312e+06,Albania,2930187,-1.469637,5.726589e+07
2,AM,22950475.0,1.740320e+03,-2.294873e+07,Armenia,2930450,-7.831130,6.948984e+07
3,AZ,14784625.0,7.530178e+02,-1.478387e+07,Azerbaijan,9827589,-1.504323,1.388962e+08
4,BA,477250.0,6.129970e+03,-4.711200e+05,Bosnia and Herzegovina,3507017,-0.134336,1.127988e+08
...,...,...,...,...,...,...,...,...
81,CA,50000.0,5.724080e+06,5.674080e+06,Canada,36624199,0.154927,2.814501e+06
82,BW,8000.0,4.214332e+02,-7.578567e+03,Botswana,2291661,-0.003307,4.151982e+06
83,BT,20000.0,3.319352e+02,-1.966806e+04,Bhutan,807610,-0.024353,4.270993e+06
84,MR,15000.0,3.441176e+01,-1.496559e+04,Mauritania,4420184,-0.003386,8.366900e+06


In [72]:
#creo una nuova colonna per il ratio tra difference e populatione_above_poverty_line
country_borr_lent["ratio_above_poverty_line"] = country_borr_lent["difference"]/country_borr_lent["population_above_poverty_line"]

In [73]:
#trovo il ratio massimo
country_borr_lent[country_borr_lent['ratio_above_poverty_line']==country_borr_lent['ratio_above_poverty_line'].max()]

,country_code,borrowed_amount,lent_amount,difference,country_name,population,ratio,population_above_poverty_line,ratio_above_poverty_line
81,CA,50000.0,5.724080e+06,5.674080e+06,Canada,36624199,0.154927,2814501.25,2.016016


# 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [74]:
#Riprendo il df loans_dates trovato al punto 2 
#Già rimossi i Nan
#Già imposto planned_expiration_time>disburse_time
#Per la risoluzione di questo punto converto il dataframe dask in un df pandas perchè dask presenta incongruenze 
#e pandas riesce a gestire efficaciemente il compito 

In [75]:
loans_dates=loans_dates.compute()

In [76]:
loans_dates=loans_dates.reset_index(drop=True)

In [77]:
loans_dates

,loan_id,disburse_time,planned_expiration_time,loan_amount
0,657307,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,125.0
1,657259,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,400.0
2,658010,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,400.0
3,659347,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,625.0
4,656933,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,425.0
...,...,...,...,...
1030022,988180,2015-11-23 08:00:00+00:00,2016-01-02 01:00:03+00:00,400.0
1030023,988213,2015-11-24 08:00:00+00:00,2016-01-02 16:40:07+00:00,300.0
1030024,989109,2015-11-13 08:00:00+00:00,2016-01-03 22:20:04+00:00,2425.0
1030025,989143,2015-11-03 08:00:00+00:00,2016-01-05 08:50:02+00:00,100.0


In [78]:
#Rimuovo il fuso orario e le ore dalle colonne datetime
loans_dates["disburse_time"] = loans_dates["disburse_time"].dt.tz_localize(None)
loans_dates["planned_expiration_time"] = loans_dates["planned_expiration_time"].dt.tz_localize(None)

In [79]:
loans_dates["disburse_time"] = loans_dates["disburse_time"].dt.normalize()
loans_dates["planned_expiration_time"] = loans_dates["planned_expiration_time"].dt.normalize()

In [80]:
loans_dates.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount
0,657307,2013-12-22,2014-02-14,125.0
1,657259,2013-12-20,2014-03-26,400.0
2,658010,2014-01-09,2014-02-15,400.0
3,659347,2014-01-17,2014-02-21,625.0
4,656933,2013-12-17,2014-02-13,425.0


In [81]:
#Ora creo due df, uno in cui compaiono i prestiti che hanno disburse_time e planned_expiration_time
#nello stesso anno e uno in cui queste date non coincidono.
same_year_loans=loans_dates[loans_dates["disburse_time"].dt.year == loans_dates["planned_expiration_time"].dt.year]
diff_year_loans=loans_dates[loans_dates["disburse_time"].dt.year != loans_dates["planned_expiration_time"].dt.year]

In [82]:
same_year_loans.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount
2,658010,2014-01-09,2014-02-15,400.0
3,659347,2014-01-17,2014-02-21,625.0
5,659605,2014-01-15,2014-02-20,350.0
6,660240,2014-01-20,2014-02-21,125.0
7,661601,2014-01-10,2014-02-25,1600.0


In [83]:
diff_year_loans.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount
0,657307,2013-12-22,2014-02-14,125.0
1,657259,2013-12-20,2014-03-26,400.0
4,656933,2013-12-17,2014-02-13,425.0
12,660363,2013-12-23,2014-02-21,1175.0
13,661165,2013-12-26,2014-03-26,300.0


## Same_year_loans

In [84]:
#Comincio a trovare l'importo dei loans per anno del df same_year_loans eseguendo una groupby sul disburse_time 
#e una sum sul loan_amount
same_year_loans_tot = same_year_loans.groupby(same_year_loans["disburse_time"].dt.year)["loan_amount"].sum().to_frame().reset_index()

In [85]:
same_year_loans_tot.head()

,disburse_time,loan_amount
0,2012,103911725.0
1,2013,98427750.0
2,2014,120644250.0
3,2015,131208475.0
4,2016,133271575.0


## Diff_year_loans

Per quanto riguarda "diff_year_loans", in cui compaiono anni diversi per disburse_time e planned_expiration_time, la risoluzione del problema si presenta ovviamente più difficoltosa.
Prendendo lo stesso esempio della consegna, vediamo come risolverlo.
Il dataset, così com'è, si presenta a noi nella seguente forma:

| loan_id | disburse_time | planned_expiration_time | loan_amount |
|---------|---------------|-------------------------|-------------|
|    1    |   2016-12-01  |        2018-30-01       |    5000     |

Per poter suddividere il loan_amount in tutti gli anni di durata del loan, devo trovare prima di tutto i giorni esatti di competenza di ogni anno:

1. Tot di giorni dal disburse_time alla fine dell'anno a cui appartiene
2. Tot di giorni degli anni successivi, prima dell'anno a cui appartiene planned_expiration_time
3. Tot di giorni dall'inizio dell'anno a cui appartiene planned_expiration_time, fino alla data effettiva di planned_expiration_time

Per arrivare ad ottenere questi risultati, dobbiamo ricavare informazioni aggiuntive e aggiungerle alla tabella iniziale, ottenendo una tabella come la seguente:

| loan_id | disburse_time | planned_expiration_time | loan_amount |  start_year  |  end_year  | days_to_end |tot_days| 
|---------|---------------|-------------------------|-------------|--------------|------------|-------------|--------|
|    1    |   2016-12-01  |        2018-30-01       |    5000     | 2016-01-01   | 2016-31-12 |     31      |  426   |
|         |               |                         |             |              |            |             |        |
|    1    |   2016-12-01  |        2018-30-01       |    5000     | 2017-01-01   | 2017-31-12 |     365     |  426   |
|         |               |                         |             |              |            |             |        |
|    1    |   2016-12-01  |        2018-30-01       |    5000     | 2018-01-01   | 2018-31-12 |     30      |  426   |


Una volta ottenute queste informazioni, basterà eseguire il calcolo come nell'esempio:


$$\large loan\hspace{0.2cm}amount\hspace{0.2cm}per\hspace{0.2cm}year = loan\hspace{0.2cm}amount \frac{days\hspace{0.2cm}to\hspace{0.2cm}end}{tot\hspace{0.2cm}days}$$

Nell'esempio:
$$ loan\hspace{0.2cm}amount\hspace{0.2cm}per\hspace{0.2cm}2016= 5000\frac{31}{31+365+30}$$

$$ loan\hspace{0.2cm}amount\hspace{0.2cm}per\hspace{0.2cm}2017= 5000\frac{365}{31+365+30}$$

$$ loan\hspace{0.2cm}amount\hspace{0.2cm}per\hspace{0.2cm}2018= 5000\frac{30}{31+365+30}$$

Procedo ad eseguire quanto spiegato:

In [86]:
#Ripeto le righe n=(anni di durata+1) volte. Il +1 va aggiunto perchè la semplice differenza darebbe come risultato 1 anno in meno

diff_year_loans = diff_year_loans.loc[diff_year_loans.index.repeat(diff_year_loans.planned_expiration_time.dt.year - diff_year_loans.disburse_time.dt.year + 1)]

In [87]:
diff_year_loans

,loan_id,disburse_time,planned_expiration_time,loan_amount
0,657307,2013-12-22,2014-02-14,125.0
0,657307,2013-12-22,2014-02-14,125.0
1,657259,2013-12-20,2014-03-26,400.0
1,657259,2013-12-20,2014-03-26,400.0
4,656933,2013-12-17,2014-02-13,425.0
...,...,...,...,...
1030024,989109,2015-11-13,2016-01-03,2425.0
1030025,989143,2015-11-03,2016-01-05,100.0
1030025,989143,2015-11-03,2016-01-05,100.0
1030026,989240,2015-11-03,2016-01-03,175.0


#### Start_year

In [88]:
#Creo la colonna start_year

diff_year_loans["start_year"] = diff_year_loans.drop_duplicates()["disburse_time"].dt.year.apply(lambda x: dt.datetime(x, 1, 1))


In [89]:
#Vado a modificarla incrementando di un anno ogni volta che la riga si ripete, 
#per ottenere la data di inizio dell'anno di ogni anno di competenza del loan
y = diff_year_loans["disburse_time"].dt.year
diff_year_loans["start_year"] = pd.to_datetime(diff_year_loans.groupby(diff_year_loans["loan_id"]).cumcount() + y, format='%Y')

In [90]:
diff_year_loans.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount,start_year
0,657307,2013-12-22,2014-02-14,125.0,2013-01-01
0,657307,2013-12-22,2014-02-14,125.0,2014-01-01
1,657259,2013-12-20,2014-03-26,400.0,2013-01-01
1,657259,2013-12-20,2014-03-26,400.0,2014-01-01
4,656933,2013-12-17,2014-02-13,425.0,2013-01-01


#### End_year

In [91]:
#Creo end_year e la popolo similmente a start_year

diff_year_loans["end_year"] = pd.to_datetime(diff_year_loans.groupby(diff_year_loans["loan_id"]).cumcount() + y, format='%Y')

In [92]:
diff_year_loans["end_year"] = diff_year_loans["start_year"].dt.year.apply(lambda x: dt.datetime(x, 12, 31))

In [93]:
#resetto indice
diff_year_loans=diff_year_loans.reset_index(drop=True)

#### Days_to_end

Ora che ho start_year ed end_year, devo popolare "days_to_end" e lo faccio sfruttando 3 diverse loc.

La logica è la seguente:

```df.loc[condizioni, "days_to_end"] = assegno valore```

Con la **prima loc** prendo in considerazione le righe che rispettano le seguenti condizioni:
1. Anno di **planned_expiration_time** > anno **end_year** (escludo così tutti gli anni a cui appartengono i planned_expiration_time, per ogni loan_id)
2. Anno di **disburse_time** = anno di **start_year** (escludo così tutti gli anni che intercorrono tra la data di disburse_time e planned_expiration_time, per ogni loan_id)


In questo modo ho estratto tutte le righe riferite all'anno in cui si verifica il disburse_time per ogni loan_id. 
A questo punto assegno a tutte le suddette righe il valore di days_to_end, in questo caso calcolato come la differenza tra end_year e disburse_time.

In [94]:
#Con questa prima loc assegno i days_to_end all'anno   
diff_year_loans.loc[(diff_year_loans["planned_expiration_time"].dt.year > diff_year_loans["end_year"].dt.year) & (diff_year_loans["disburse_time"].dt.year == diff_year_loans["start_year"].dt.year), "days_to_end"] = (diff_year_loans["end_year"] - diff_year_loans["disburse_time"]).dt.days +1

In [95]:
#Stampo la tabella risultante per mostrare che in tutte le righe escluse dalle condizioni imposte compaiono valori NaN 
#Al contrario, in tutte le corrispondenti all'anno di disburse_time dei loans, compare il valore days_to_end
diff_year_loans

,loan_id,disburse_time,planned_expiration_time,loan_amount,start_year,end_year,days_to_end
0,657307,2013-12-22,2014-02-14,125.0,2013-01-01,2013-12-31,10.0
1,657307,2013-12-22,2014-02-14,125.0,2014-01-01,2014-12-31,NaN
2,657259,2013-12-20,2014-03-26,400.0,2013-01-01,2013-12-31,12.0
3,657259,2013-12-20,2014-03-26,400.0,2014-01-01,2014-12-31,NaN
4,656933,2013-12-17,2014-02-13,425.0,2013-01-01,2013-12-31,15.0
...,...,...,...,...,...,...,...
302198,989109,2015-11-13,2016-01-03,2425.0,2016-01-01,2016-12-31,NaN
302199,989143,2015-11-03,2016-01-05,100.0,2015-01-01,2015-12-31,59.0
302200,989143,2015-11-03,2016-01-05,100.0,2016-01-01,2016-12-31,NaN
302201,989240,2015-11-03,2016-01-03,175.0,2015-01-01,2015-12-31,59.0


Similmente, la **seconda loc** prende in considerazione le righe che rispettano le seguenti condizioni:

1. Anno di **planned_expiration_time** > anno **end_year** (escludo così tutti gli anni a cui appartengono i planned_expiration_time, per ogni loan_id)
2. Anno di **disburse_time** ≠ anno di **start_year** (escludo così tutti gli anni a cui appartengono i disburse_time, per ogni loan_id) 


In questo modo ho estratto tutte le righe riferite agli anni che intercorrono tra la data di disburse_time e planned_expiration_time di ogni loan_id.
A questo punto assegno a tutte le suddette righe il valore di days_to_end, in questo caso calcolato come la differenza tra end_year e start_year.

In [96]:
#Implemento la logica che ho descritto poco sopra

diff_year_loans.loc[(diff_year_loans["planned_expiration_time"].dt.year > diff_year_loans["end_year"].dt.year) & (diff_year_loans["disburse_time"].dt.year != diff_year_loans["start_year"].dt.year), "days_to_end"] = (diff_year_loans["end_year"] - diff_year_loans["start_year"]).dt.days +1

Infine, la **terza loc** prende in considerazione le righe che rispettano la seguente condizione:
1. Anno di **planned_expiration_time** > anno **end_year** della riga in oggetto 


In questo modo ho estratto le righe rimanenti, ovvero quelle riferite agli anni a cui appartengono i planned_expiration_time.
A questo punto assegno a tutte le suddette righe il valore di days_to_end, in questo caso calcolato come la differenza tra planned_expiration_time e start_year.

In [97]:
diff_year_loans.loc[(diff_year_loans["planned_expiration_time"].dt.year == diff_year_loans["end_year"].dt.year), "days_to_end"] = (diff_year_loans["planned_expiration_time"] - diff_year_loans["start_year"]).dt.days +1

In [98]:
diff_year_loans.head()

,loan_id,disburse_time,planned_expiration_time,loan_amount,start_year,end_year,days_to_end
0,657307,2013-12-22,2014-02-14,125.0,2013-01-01,2013-12-31,10.0
1,657307,2013-12-22,2014-02-14,125.0,2014-01-01,2014-12-31,45.0
2,657259,2013-12-20,2014-03-26,400.0,2013-01-01,2013-12-31,12.0
3,657259,2013-12-20,2014-03-26,400.0,2014-01-01,2014-12-31,85.0
4,656933,2013-12-17,2014-02-13,425.0,2013-01-01,2013-12-31,15.0


#### Tot_days

In [99]:
#Calcolo tot_days come la differenza tra planned_expiration_time e disburse_time 

diff_year_loans["tot_days"] = (diff_year_loans["planned_expiration_time"] - diff_year_loans["disburse_time"]).dt.days +1

#### Loan_amount_per_year

Ora ho tutti i dati di cui ho bisogno per poter calcolare il loan_amount_per_year secondo la formula introdotta all'inizio e nell'esempio:

$$\large loan\hspace{0.2cm}amount\hspace{0.2cm}per\hspace{0.2cm}year = loan\hspace{0.2cm}amount \frac{days\hspace{0.2cm}to\hspace{0.2cm}end}{tot\hspace{0.2cm}days}$$



In [100]:
diff_year_loans["loan_amount_per_year"] = (diff_year_loans["loan_amount"]* diff_year_loans["days_to_end"])/diff_year_loans["tot_days"]

In [101]:
#Creo un nuovo dataframe in cui eseguo una groupby sullo start_year e una somma dei loan_amount_per_year

diff_year_loans_tot = diff_year_loans.groupby(diff_year_loans["start_year"].dt.year)["loan_amount_per_year"].sum().to_frame().reset_index()
display(diff_year_loans_tot)

,start_year,loan_amount_per_year
0,2011,6.614928e+05
1,2012,9.251024e+06
2,2013,2.512557e+07
3,2014,3.001850e+07
4,2015,2.394231e+07
5,2016,2.081552e+07
6,2017,2.046922e+07
7,2018,4.877086e+06


#### Unisco same_year_loans e diff_year_loans

In [102]:
#Faccio una merge dei dataframe same_year_loans_tot e diff_year_loans_tot
#Si tratta di una outer join perchè il df diff_year_loans_tot comprende un anno in più che va inserito nel df finale

tot = pd.merge(diff_year_loans_tot, same_year_loans_tot, left_on ="start_year", right_on="disburse_time", how="outer")

In [103]:
display(tot)

,start_year,loan_amount_per_year,disburse_time,loan_amount
0,2011,6.614928e+05,NaN,NaN
1,2012,9.251024e+06,2012.0,103911725.0
2,2013,2.512557e+07,2013.0,98427750.0
3,2014,3.001850e+07,2014.0,120644250.0
4,2015,2.394231e+07,2015.0,131208475.0
5,2016,2.081552e+07,2016.0,133271575.0
6,2017,2.046922e+07,2017.0,144870625.0
7,2018,4.877086e+06,2018.0,85300.0


In [104]:
#Proprio perchè il df diff_year_loans_tot comprendeva un anno in più, nel fare la merge compaiono dei valori Nan corrispondenti a quell'anno (2011)
#Riempio i valori vuoti con 0, in quanto potrebbero creare problemi con le successive somme
#Ad esempio, se in una riga avessimo "X" = 50 e "Y =  NaN", la loro somma ci darebbe NaN
#Noi invece vogliamo che la loro somma sia = 50. 

tot[["loan_amount_per_year" , "loan_amount"]] = tot[["loan_amount_per_year" , "loan_amount"]].fillna(0)
tot["start_year"] = tot["start_year"].fillna(tot["disburse_time"])
tot["disburse_time"] = tot["disburse_time"].fillna(tot["start_year"])

In [105]:
tot

,start_year,loan_amount_per_year,disburse_time,loan_amount
0,2011,6.614928e+05,2011.0,0.0
1,2012,9.251024e+06,2012.0,103911725.0
2,2013,2.512557e+07,2013.0,98427750.0
3,2014,3.001850e+07,2014.0,120644250.0
4,2015,2.394231e+07,2015.0,131208475.0
5,2016,2.081552e+07,2016.0,133271575.0
6,2017,2.046922e+07,2017.0,144870625.0
7,2018,4.877086e+06,2018.0,85300.0


In [106]:
#Sommo le due colonne degli ammontare dei due dataframe in una sola colonna

tot["total_per_year"] = tot["loan_amount_per_year"] + tot["loan_amount"]
tot = tot[["start_year", "total_per_year"]]

In [107]:
tot

,start_year,total_per_year
0,2011,6.614928e+05
1,2012,1.131627e+08
2,2013,1.235533e+08
3,2014,1.506627e+08
4,2015,1.551508e+08
5,2016,1.540871e+08
6,2017,1.653398e+08
7,2018,4.962386e+06
